In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import stem
from nltk.corpus import stopwords
import string
from nltk import *
from six import u

In [ ]:
data = pd.read_csv('Movie_Summary.csv', encoding='utf-8')

In [ ]:
def preprocess(text):
# Remove punctuation, stopword and then stemmering
    stop = set(stopwords.words("english"))
    stemmer = stem.PorterStemmer()
    punctuation = set(string.punctuation)


    doc = [stemmer.stem(word) for word in word_tokenize(text) if (word not in punctuation) and (word not in stop)]

    doc = ' '.join(w for w in doc)
    return doc

In [ ]:
data["Summary"] = data["Summary"].apply(preprocess)
summaries = data['Summary'].as_matrix()
tfidfVect = TfidfVectorizer()
tfidf = tfidfVect.fit_transform(summaries)

# TODO METTRE LA TRANSFORMATION DES LABELS

In [ ]:
X = tfidf

In [ ]:
def _dump_svmlight(X, f):
    X_is_sp = int(hasattr(X, "tocsr"))

    if X.dtype.kind == 'i':
        value_pattern = u("%d:%.4d")
    else:
        value_pattern = u("%d:%d")

    line_pattern = u("%s")

    line_pattern += u(" %s\n")

    for i in range(X.shape[0]):
        if X_is_sp:
            span = slice(X.indptr[i], X.indptr[i + 1])
            row = zip(X.indices[span], X.data[span])
        else:
            nz = X[i] != 0
            row = zip(np.where(nz)[0], X[i, nz])

        s = " ".join(value_pattern % (j, int(x*9999)) for j, x in row)


        f.write((s+"\n").encode('ascii'))

In [ ]:
f = 'movies.data'
with open(f, "wb") as f:
    _dump_svmlight(X, f)